Tema ML Etapa 1

In [ ]:
!pip install sktime
!pip install pyts
!pip install seaborn
!pip install tsfresh
!pip install xgboost

In [ ]:
import zipfile
import pandas as pd
import os

import sktime
from sktime.datasets import load_from_tsfile
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingClassifier

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from xgboost import XGBClassifier

from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
import scipy.stats as stats
from scipy.stats import skew, kurtosis, iqr

In [ ]:
# Dezarhivez setul de date Racket
with zipfile.ZipFile('RacketSports.zip', 'r') as zip_ref:
    zip_ref.extractall('RacketSports')

# Dezarhivez setul de date ECG
with zipfile.ZipFile('ECG.zip', 'r') as zip_ref:
    zip_ref.extractall('ECG')

Crearea seturilor de date

In [ ]:
# Racket
# Incarcă setul de date de antrenare
racket_train_x, racket_train_y = load_from_tsfile("RacketSports/RacketSports_TRAIN.ts")

# Incarcă setul de date de test
racket_test_x, racket_test_y = load_from_tsfile("RacketSports/RacketSports_TEST.ts")

# Convert to NumPy array
racket_train_x = np.array(racket_train_x)
racket_train_y = np.array(racket_train_y)
racket_test_x = np.array(racket_test_x)
racket_test_y = np.array(racket_test_y)

# Mitbih
# Incarcă setul de date de antrenare
train_data = pd.read_csv("ECG/mitbih_train.csv", header=None)
mitbih_train_x = train_data.iloc[:, :-1]
mitbih_train_y = train_data.iloc[:, -1]

# Incarcă setul de date de test
test_data = pd.read_csv("ECG/mitbih_test.csv", header=None)
mitbih_test_x = test_data.iloc[:, :-1]
mitbih_test_y = test_data.iloc[:, -1]

# Ptbdb
# Incarcă setul de date normal
data_set_normal = pd.read_csv("ECG/ptbdb_normal.csv", header=None)
x_normal = data_set_normal.iloc[:, :-1]
y_normal = data_set_normal.iloc[:, -1]

# Incarcă setul de date de anormal
data_set_anormal = pd.read_csv("ECG/ptbdb_abnormal.csv", header=None)
x_anormal = data_set_anormal.iloc[:, :-1]
y_anormal = data_set_anormal.iloc[:, -1]

# Concatenăm seturile de date de antrenare și testare
x_pdb = pd.concat([x_normal, x_anormal])
y_pdb = pd.concat([y_normal, y_anormal])

# Impărțim seturile de date în seturi de antrenare și testare
x_pdb_train, x_pdb_test, y_pdb_train, y_pdb_test = train_test_split(x_pdb, y_pdb, test_size=0.3, random_state=42)

3.1.1

In [ ]:
# Racket
# Definim un dataframe din setul de date de antrenare
racket_train_df = pd.DataFrame(racket_train_y, columns=["label"])

# Definim un dataframe din setul de date de test
racket_test_df = pd.DataFrame(racket_test_y, columns=["label"])

# Creăm două axe separate
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(25, 6))

# Afișăm graficul frecvenței de apariție a fiecărei etichete în setul de date de antrenare
sns.countplot(x="label", data=racket_train_df, ax=ax1)
ax1.set_title("Frecvența de apariție a fiecărei etichete în setul de antrenare - Racket")

# Afișăm graficul frecvenței de apariție a fiecărei etichete în setul de date de test
sns.countplot(x="label", data=racket_test_df, ax=ax2)
ax2.set_title("Frecvența de apariție a fiecărei etichete în setul de testare - Racket")

plt.show()

In [ ]:
# Mitbih
# Crearea unui dataframe cu datele de antrenare
train_df = pd.concat([mitbih_train_x, mitbih_train_y], axis=1)
train_df.columns = [f"feat_{i}" for i in range(187)] + ["label"]

# Crearea unui dataframe cu datele de test
test_df = pd.concat([mitbih_test_x, mitbih_test_y], axis=1)
test_df.columns = [f"feat_{i}" for i in range(187)] + ["label"]

# Creăm două axe separate
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 6))

# Afișăm graficul frecvenței de apariție a fiecărei etichete în setul de date de antrenare
sns.countplot(data=train_df, x="label", ax=ax1)
ax1.set_title("Frecvența de apariție a fiecărei etichete în setul de antrenare - Mitbih")

# Afișăm graficul frecvenței de apariție a fiecărei etichete în setul de date de test
sns.countplot(data=test_df, x="label", ax=ax2)
ax2.set_title("Frecvența de apariție a fiecărei etichete în setul de testare - Mitbih")

# Crearea unui countplot pentru datele de antrenare
plt.show()

In [ ]:
# Ptdbd
# Crearea unui dataframe cu datele de antrenare
train_df = pd.concat([x_pdb_train, y_pdb_train], axis=1)
train_df.columns = [f"feat_{i}" for i in range(187)] + ["label"]

# Crearea unui dataframe cu datele de test
test_df = pd.concat([x_pdb_test, y_pdb_test], axis=1)
test_df.columns = [f"feat_{i}" for i in range(187)] + ["label"]

# Creăm două axe separate
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 6))

# Afișăm graficul frecvenței de apariție a fiecărei etichete în setul de date de antrenare
sns.countplot(data=train_df, x="label", ax=ax1)
ax1.set_title("Frecvența de apariție a fiecărei etichete în setul de antrenare - Ptdbd")

# Afișăm graficul frecvenței de apariție a fiecărei etichete în setul de date de test
sns.countplot(data=test_df, x="label", ax=ax2)
ax2.set_title("Frecvența de apariție a fiecărei etichete în setul de testare - Ptdbd")

# Crearea unui countplot pentru datele de antrenare
plt.show()

3.1.2.1

In [ ]:
# Identificăm un exemplu de serie pentru fiecare tip de acțiune
unique_actions = np.unique(racket_train_y)
example_indices = [np.where(racket_train_y == action)[0][0] for action in unique_actions]

# Afișăm graficele pentru fiecare acțiune
for i, example_idx in enumerate(example_indices):
    action = racket_train_y[example_idx]
    example_series = racket_train_x[example_idx]

    # a) Afișăm valorile de accelerometru pe dimensiunile x, y și z pe același grafic
    plt.figure(figsize=(12, 4))
    plt.title(f"Acțiunea {action}: Valorile accelerometrului")
    plt.plot(example_series[0], label="Accelerometru X")
    plt.plot(example_series[1], label="Accelerometru Y")
    plt.plot(example_series[2], label="Accelerometru Z")
    plt.legend()
    plt.show()

    # b) Afișăm valorile de giroscop pe dimensiunile x, y și z pe același grafic
    plt.figure(figsize=(12, 4))
    plt.title(f"Acțiunea {action}: Valorile giroscopului")
    plt.plot(example_series[3], label="Giroscop X")
    plt.plot(example_series[4], label="Giroscop Y")
    plt.plot(example_series[5], label="Giroscop Z")
    plt.legend()
    plt.show()

3.1.2.2

In [ ]:
# Mitbih
# Luam actiunile unice din setul de training
mitbih_train_classes = np.unique(mitbih_train_y)

# Afisam câte un exemplu de serie pentru fiecare categorie de aritmie
for c in mitbih_train_classes:
    index = np.where(mitbih_train_y == c)[0][0]
    plt.plot(mitbih_train_x.iloc[index])
    plt.title(f"Clasa {int(c)}")
    plt.show()

In [ ]:
# Ptdbd
# Luam actiunile unice din setul de training
mitbih_train_classes = np.unique(y_pdb_train)

# Afisam câte un exemplu de serie pentru fiecare categorie de aritmie
for c in mitbih_train_classes:
    index = np.where(y_pdb_train == c)[0][0]
    plt.plot(x_pdb_train.iloc[index])
    plt.title(f"Clasa {int(c)}")
    plt.show()

3.1.2.3

In [ ]:
# Mitbih
# Combinați seturile de date de antrenament și testare
mitbih_x = pd.concat([mitbih_train_x, mitbih_test_x])
mitbih_y = pd.concat([mitbih_train_y, mitbih_test_y])

# Calculează media și deviația standard pentru fiecare clasă de aritmie
classes = np.unique(mitbih_y)
means = []
stds = []
for c in classes:
    c_data = mitbih_x[mitbih_y == c]
    c_mean = np.mean(c_data, axis=0)
    c_std = np.std(c_data, axis=0)
    means.append(c_mean)
    stds.append(c_std)

# Afișați graficul mediei și deviației standard per unitate de timp pentru fiecare clasă de aritmie
for i in range(len(classes)):
    plt.plot(means[i], label=f"{classes[i]} mean")
    plt.plot(stds[i], label=f"{classes[i]} std")
plt.legend()
plt.title("Media și deviația standard per unitate de timp pentru fiecare clasă de aritmie - Mitbih")
plt.show()

In [ ]:
# Ptdbd
# Combinați seturile de date de antrenament și testare
pdb_x = pd.concat([x_pdb_train, x_pdb_test])
pdb_y = pd.concat([y_pdb_train, y_pdb_test])

# Calculează media și deviația standard pentru fiecare clasă de aritmie
classes = np.unique(pdb_y)
means = []
stds = []
for c in classes:
    c_data = pdb_x[pdb_y == c]
    c_mean = np.mean(c_data, axis=0)
    c_std = np.std(c_data, axis=0)
    means.append(c_mean)
    stds.append(c_std)

# Afișați graficul mediei și deviației standard per unitate de timp pentru fiecare clasă de aritmie
for i in range(len(classes)):
    plt.plot(means[i], label=f"{classes[i]} mean")
    plt.plot(stds[i], label=f"{classes[i]} std")
plt.legend()
plt.title("Media și deviația standard per unitate de timp pentru fiecare clasă de aritmie - Ptdbd")
plt.show()

3.1.2.4

In [ ]:
# Creăm un DataFrame cu coloanele corespunzătoare
columns = ["Action", "Sensor", "Axis", "Value"]
data = []

for i, series in enumerate(racket_train_x):
    action = racket_train_y[i]
    for sensor_idx, sensor_name in enumerate(["Accelerometru", "Giroscop"]):
        for axis_idx, axis_name in enumerate(["X", "Y", "Z"]):
            values = series[sensor_idx * 3 + axis_idx]
            for value in values:
                data.append([action, sensor_name, axis_name, value])

df = pd.DataFrame(data, columns=columns)

# Afișăm distribuția valorilor per axă și acțiune
sns.set(style="whitegrid")
g = sns.catplot(x="Action", y="Value", hue="Axis", col="Sensor",
                data=df, kind="box", height=6, aspect=1.5)
g.fig.subplots_adjust(top=0.85)
g.fig.suptitle("Distribuția valorilor per axă de accelerometru și giroscop în parte / per acțiune")
plt.show()

3.2.1

RandomForest

In [ ]:
# Racket
def resize_series(series, new_length):
    old_length = len(series[0])
    x_old = np.linspace(0, 1, old_length)
    x_new = np.linspace(0, 1, new_length)
    resized_series = []

    for feature in series:
        interpolator = interp1d(x_old, feature, kind='linear')
        resized_series.append(interpolator(x_new))

    return np.array(resized_series)

def prepare_data(train_x, train_y, test_x, test_y, new_length=100):
    train_x = [resize_series(series, new_length).flatten() for series in train_x]
    test_x = [resize_series(series, new_length).flatten() for series in test_x]
    return np.array(train_x), np.array(train_y), np.array(test_x), np.array(test_y)

In [ ]:
# Racket
# Pregătirea setului de date
train_x, train_y, test_x, test_y = prepare_data(racket_train_x, racket_train_y, racket_test_x, racket_test_y)

# Antrenarea modelului RandomForest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(train_x, train_y)

# Evaluarea modelului pe setul de test
test_preds = rf_model.predict(test_x)
accuracy = accuracy_score(test_y, test_preds)
report = classification_report(test_y, test_preds)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

In [ ]:
# Racket-hiperparametrii
# Pregătirea setului de date
train_x, train_y, test_x, test_y = prepare_data(racket_train_x, racket_train_y, racket_test_x, racket_test_y, new_length=100)

# Definirea hiper-parametrilor pentru RandomForest
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Crearea modelului RandomForest
rf_model = RandomForestClassifier(random_state=42)

# Grid Search cu Cross Validation pentru a găsi cei mai buni hiper-parametri
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(train_x, train_y)

# Afișarea celor mai buni hiper-parametri găsiți
print("Cei mai buni hiper-parametri pentru RandomForest (RacketSports):")
print(grid_search_rf.best_params_)

# Evaluarea modelului cu cei mai buni hiper-parametri pe setul de test
best_rf_model = grid_search_rf.best_estimator_
test_preds_best_rf = best_rf_model.predict(test_x)
accuracy_best_rf = accuracy_score(test_y, test_preds_best_rf)
report_best_rf = classification_report(test_y, test_preds_best_rf)

print(f"Accuracy cu cei mai buni hiper-parametri RandomForest (RacketSports): {accuracy_best_rf}")
print("Classification Report cu cei mai buni hiper-parametri RandomForest (RacketSports):")
print(report_best_rf)

In [ ]:
# Mitbih
# Antrenarea modelului RandomForest
rf_model_mitbih = RandomForestClassifier(random_state=42)
rf_model_mitbih.fit(mitbih_train_x, mitbih_train_y)

# Evaluarea modelului pe setul de test
mitbih_test_preds = rf_model_mitbih.predict(mitbih_test_x)
mitbih_accuracy = accuracy_score(mitbih_test_y, mitbih_test_preds)
mitbih_report = classification_report(mitbih_test_y, mitbih_test_preds)

print(f"MITBIH Accuracy: {mitbih_accuracy}")
print("MITBIH Classification Report:")
print(mitbih_report)

In [ ]:
# Mitbih-hiperparametrii
# Definirea hiper-parametrilor pentru RandomForest
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Crearea modelului RandomForest
rf_model = RandomForestClassifier(random_state=42)

# Grid Search cu Cross Validation pentru a găsi cei mai buni hiper-parametri
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(mitbih_train_x, mitbih_train_y)

# Afișarea celor mai buni hiper-parametri găsiți
print("Cei mai buni hiper-parametri pentru RandomForest (MITBIH):")
print(grid_search_rf.best_params_)

# Evaluarea modelului cu cei mai buni hiper-parametri pe setul de test
best_rf_model = grid_search_rf.best_estimator_
test_preds_best_rf = best_rf_model.predict(mitbih_test_x)
accuracy_best_rf = accuracy_score(mitbih_test_y, test_preds_best_rf)
report_best_rf = classification_report(mitbih_test_y, test_preds_best_rf)

print(f"Accuracy cu cei mai buni hiper-parametri RandomForest (MITBIH): {accuracy_best_rf}")
print("Classification Report cu cei mai buni hiper-parametri RandomForest (MITBIH):")
print(report_best_rf)

In [ ]:
# Ptdbd
# Antrenarea modelului RandomForest
rf_model_ptbdb = RandomForestClassifier(random_state=42)
rf_model_ptbdb.fit(x_pdb_train, y_pdb_train)

# Evaluarea modelului pe setul de test
ptbdb_test_preds = rf_model_ptbdb.predict(x_pdb_test)
ptbdb_accuracy = accuracy_score(y_pdb_test, ptbdb_test_preds)
ptbdb_report = classification_report(y_pdb_test, ptbdb_test_preds)

print(f"PTBDB Accuracy: {ptbdb_accuracy}")
print("PTBDB Classification Report:")
print(ptbdb_report)

In [ ]:
# Ptdbd-hiperparametrii
# Definirea hiper-parametrilor pentru RandomForest
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Crearea modelului RandomForest
rf_model = RandomForestClassifier(random_state=42)

# Grid Search cu Cross Validation pentru a găsi cei mai buni hiper-parametri
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(x_pdb_train, y_pdb_train)

# Afișarea celor mai buni hiper-parametri găsiți
print("Cei mai buni hiper-parametri pentru RandomForest (PTBDB):")
print(grid_search_rf.best_params_)

# Evaluarea modelului cu cei mai buni hiper-parametri pe setul de test
best_rf_model = grid_search_rf.best_estimator_
test_preds_best_rf = best_rf_model.predict(x_pdb_test)
accuracy_best_rf = accuracy_score(y_pdb_test, test_preds_best_rf)
report_best_rf = classification_report(y_pdb_test, test_preds_best_rf)

print(f"Accuracy cu cei mai buni hiper-parametri RandomForest (PTBDB): {accuracy_best_rf}")
print("Classification Report cu cei mai buni hiper-parametri RandomForest (PTBDB):")
print(report_best_rf)

GradientBoosted Trees

In [ ]:
# Racket
# Pregătirea setului de date
train_x, train_y, test_x, test_y = prepare_data(racket_train_x, racket_train_y, racket_test_x, racket_test_y, new_length=100)

# Codificarea etichetelor ca numere întregi
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
test_y_encoded = label_encoder.transform(test_y)

# Antrenarea modelului XGBoost
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(train_x, train_y_encoded)

# Evaluarea modelului pe setul de test
test_preds_xgb = xgb_model.predict(test_x)
accuracy_xgb = accuracy_score(test_y_encoded, test_preds_xgb)
report_xgb = classification_report(test_y_encoded, test_preds_xgb)

print(f"XGBoost Accuracy: {accuracy_xgb}")
print("XGBoost Classification Report:")
print(report_xgb)

In [ ]:
# Racket-hiperparametrii
# Pregătiți seturile de date
racket_train_x_prep, racket_train_y_prep, racket_test_x_prep, racket_test_y_prep = prepare_data(
    racket_train_x, racket_train_y, racket_test_x, racket_test_y, new_length=100
)

# Definiți hiper-parametrii pentru XGBClassifier
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 6, 10],
    "learning_rate": [0.01, 0.1, 0.2],
}

# Creați o instanță a XGBClassifier
xgb_clf = XGBClassifier(random_state=42)

# Creați obiectul GridSearchCV
grid_search = GridSearchCV(xgb_clf, param_grid, cv=5, scoring="accuracy")

# Potriviți obiectul GridSearchCV pe datele de antrenament
grid_search.fit(racket_train_x_prep, racket_train_y_prep)

# Găsiți cei mai buni parametri și scorul corespunzător
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_score}")

# Antrenați XGBClassifier cu cei mai buni parametri pe întregul set de antrenament
xgb_clf_best = XGBClassifier(**best_params, random_state=42)
xgb_clf_best.fit(racket_train_x_prep, racket_train_y_prep)

# Evaluați modelul pe setul de testare
test_preds_xgb = xgb_clf_best.predict(racket_test_x_prep)
accuracy_xgb = accuracy_score(racket_test_y_prep, test_preds_xgb)
report_xgb = classification_report(racket_test_y_prep, test_preds_xgb)

print(f"XGBClassifier Accuracy: {accuracy_xgb}")
print("XGBClassifier Classification Report:")
print(report_xgb)

In [ ]:
# Mitbih
# Antrenarea modelului XGBoost
xgb_model_mitbih = xgb.XGBClassifier(random_state=42)
xgb_model_mitbih.fit(mitbih_train_x, mitbih_train_y)

# Evaluarea modelului pe setul de test
mitbih_test_preds_xgb = xgb_model_mitbih.predict(mitbih_test_x)
mitbih_accuracy_xgb = accuracy_score(mitbih_test_y, mitbih_test_preds_xgb)
mitbih_report_xgb = classification_report(mitbih_test_y, mitbih_test_preds_xgb)

print(f"MITBIH XGBoost Accuracy: {mitbih_accuracy_xgb}")
print("MITBIH XGBoost Classification Report:")
print(mitbih_report_xgb)

In [ ]:
# Mitbih-hiperparametrii
# Definirea parametrilor pentru Grid Search
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [3, 5, 10, 15],
    'learning_rate': [0.01, 0.1, 0.2, 0.3]
}

# Crearea modelului XGBClassifier
xgb_model = xgb.XGBClassifier(use_label_encoder=False, objective='multi:softmax', random_state=42, eval_metric='mlogloss', n_jobs=-1)

# Crearea obiectului GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring=make_scorer(accuracy_score), cv=5, verbose=1, n_jobs=-1)

# Antrenarea modelului folosind Grid Search
grid_search.fit(mitbih_train_x, mitbih_train_y)

# Afișarea celor mai buni hiper-parametri găsiți
print("Cei mai buni hiper-parametri găsiți:")
print(grid_search.best_params_)

# Evaluarea modelului pe setul de test
best_xgb = grid_search.best_estimator_
predictions = best_xgb.predict(mitbih_test_x)
accuracy = accuracy_score(mitbih_test_y, predictions)

print("Acuratețea modelului pe setul de test:", accuracy)

In [ ]:
# Ptdbd
# Antrenarea modelului XGBoost
xgb_model_ptbdb = xgb.XGBClassifier(random_state=42)
xgb_model_ptbdb.fit(x_pdb_train, y_pdb_train)

# Evaluarea modelului pe setul de test
ptbdb_test_preds_xgb = xgb_model_ptbdb.predict(x_pdb_test)
ptbdb_accuracy_xgb = accuracy_score(y_pdb_test, ptbdb_test_preds_xgb)
ptbdb_report_xgb = classification_report(y_pdb_test, ptbdb_test_preds_xgb)

print(f"PTBDB XGBoost Accuracy: {ptbdb_accuracy_xgb}")
print("PTBDB XGBoost Classification Report:")
print(ptbdb_report_xgb)

SVM 

In [ ]:
# Racket
# Pregătirea setului de date
train_x, train_y, test_x, test_y = prepare_data(racket_train_x, racket_train_y, racket_test_x, racket_test_y, new_length=100)

# Antrenarea modelului SVM
svm_model = SVC(random_state=42)
svm_model.fit(train_x, train_y)

# Evaluarea modelului pe setul de test
test_preds_svm = svm_model.predict(test_x)
accuracy_svm = accuracy_score(test_y, test_preds_svm)
report_svm = classification_report(test_y, test_preds_svm)

print(f"SVM Accuracy: {accuracy_svm}")
print("SVM Classification Report:")
print(report_svm)

In [ ]:
# Racket-hiperparametrii
# Pregătirea setului de date
train_x, train_y, test_x, test_y = prepare_data(racket_train_x, racket_train_y, racket_test_x, racket_test_y, new_length=100)

# Definirea hiper-parametrilor pentru SVM
param_grid_svm = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100]
}

# Crearea modelului SVM
svm_model = SVC(random_state=42)

# Grid Search cu Cross Validation pentru a găsi cei mai buni hiper-parametri
grid_search_svm = GridSearchCV(svm_model, param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(train_x, train_y)

# Afișarea celor mai buni hiper-parametri găsiți
print("Cei mai buni hiper-parametri pentru SVM (RacketSports):")
print(grid_search_svm.best_params_)

# Evaluarea modelului cu cei mai buni hiper-parametri pe setul de test
best_svm_model = grid_search_svm.best_estimator_
test_preds_best_svm = best_svm_model.predict(test_x)
accuracy_best_svm = accuracy_score(test_y, test_preds_best_svm)
report_best_svm = classification_report(test_y, test_preds_best_svm)

print(f"Accuracy cu cei mai buni hiper-parametri SVM (RacketSports): {accuracy_best_svm}")
print("Classification Report cu cei mai buni hiper-parametri SVM (RacketSports):")
print(report_best_svm)

In [ ]:
# Mitbih
# Pregătirea și standardizarea setului de date
scaler = StandardScaler()
mitbih_train_x_std = scaler.fit_transform(mitbih_train_x)
mitbih_test_x_std = scaler.transform(mitbih_test_x)

# Antrenarea modelului SVM
svm_model_mitbih = SVC(random_state=42)
svm_model_mitbih.fit(mitbih_train_x_std, mitbih_train_y)

# Evaluarea modelului pe setul de test
test_preds_svm_mitbih = svm_model_mitbih.predict(mitbih_test_x_std)
accuracy_svm_mitbih = accuracy_score(mitbih_test_y, test_preds_svm_mitbih)
report_svm_mitbih = classification_report(mitbih_test_y, test_preds_svm_mitbih)

print(f"SVM Accuracy (Mitbih): {accuracy_svm_mitbih}")
print("SVM Classification Report (Mitbih):")
print(report_svm_mitbih)

In [ ]:
# Mitbih-hiperparametrii
# Efectuarea standardizării datelor
scaler = StandardScaler()
mitbih_train_x_std = scaler.fit_transform(mitbih_train_x)
mitbih_test_x_std = scaler.transform(mitbih_test_x)

# Definirea hiper-parametrilor pentru SVM
param_grid_svm = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100]
}

# Crearea modelului SVM
svm_model = SVC(random_state=42)

# Grid Search cu Cross Validation pentru a găsi cei mai buni hiper-parametri
grid_search_svm = GridSearchCV(svm_model, param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(mitbih_train_x_std, mitbih_train_y)

# Afișarea celor mai buni hiper-parametri găsiți
print("Cei mai buni hiper-parametri pentru SVM (Mitbih):")
print(grid_search_svm.best_params_)

# Evaluarea modelului cu cei mai buni hiper-parametri pe setul de test
best_svm_model = grid_search_svm.best_estimator_
test_preds_best_svm = best_svm_model.predict(mitbih_test_x_std)
accuracy_best_svm = accuracy_score(mitbih_test_y, test_preds_best_svm)
report_best_svm = classification_report(mitbih_test_y, test_preds_best_svm)

print(f"Accuracy cu cei mai buni hiper-parametri SVM (Mitbih): {accuracy_best_svm}")
print("Classification Report cu cei mai buni hiper-parametri SVM (Mitbih):")
print(report_best_svm)

In [ ]:
# Ptdbd
# Pregătirea și standardizarea setului de date
scaler = StandardScaler()
x_pdb_train_std = scaler.fit_transform(x_pdb_train)
x_pdb_test_std = scaler.transform(x_pdb_test)

# Antrenarea modelului SVM
svm_model_pdb = SVC(random_state=42)
svm_model_pdb.fit(x_pdb_train_std, y_pdb_train)

# Evaluarea modelului pe setul de test
test_preds_svm_pdb = svm_model_pdb.predict(x_pdb_test_std)
accuracy_svm_pdb = accuracy_score(y_pdb_test, test_preds_svm_pdb)
report_svm_pdb = classification_report(y_pdb_test, test_preds_svm_pdb)

print(f"SVM Accuracy (Ptbdb): {accuracy_svm_pdb}")
print("SVM Classification Report (Ptbdb):")
print(report_svm_pdb)

In [ ]:
# Ptdbd-hiperparametrii
# Definirea hiper-parametrilor pentru SVM
param_grid_svm = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100]
}

# Crearea modelului SVM
svm_model = SVC(random_state=42)

# Grid Search cu Cross Validation pentru a găsi cei mai buni hiper-parametri
grid_search_svm = GridSearchCV(svm_model, param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(x_pdb_train_std, y_pdb_train)

# Afișarea celor mai buni hiper-parametri găsiți
print("Cei mai buni hiper-parametri pentru SVM (Ptbdb):")
print(grid_search_svm.best_params_)

# Evaluarea modelului cu cei mai buni hiper-parametri pe setul de test
best_svm_model = grid_search_svm.best_estimator_
test_preds_best_svm = best_svm_model.predict(x_pdb_test_std)
accuracy_best_svm = accuracy_score(y_pdb_test, test_preds_best_svm)
report_best_svm = classification_report(y_pdb_test, test_preds_best_svm)

print(f"Accuracy cu cei mai buni hiper-parametri SVM (Ptbdb): {accuracy_best_svm}")
print("Classification Report cu cei mai buni hiper-parametri SVM (Ptbdb):")
print(report_best_svm)

In [ ]:
def extract_features(window):
    features = {}
    
    window = np.array(window).T  # Transpuneți fereastra înainte de a extrage axele

    # Atribute statistice pentru fiecare axă
    for i, axis in enumerate(['x', 'y', 'z']):
        axis_data = window[i]
        prefix = f'axis_{i}_'
        
        features[prefix + 'mean'] = np.mean(axis_data)
        features[prefix + 'std'] = np.std(axis_data)
        features[prefix + 'mad'] = np.mean(np.abs(axis_data - np.mean(axis_data)))
        features[prefix + 'min'] = np.min(axis_data)
        features[prefix + 'max'] = np.max(axis_data)
        features[prefix + 'range'] = np.max(axis_data) - np.min(axis_data)
        features[prefix + 'median'] = np.median(axis_data)
        features[prefix + 'mad_from_median'] = np.mean(np.abs(axis_data - np.median(axis_data)))
        features[prefix + 'iqr'] = iqr(axis_data)
        features[prefix + 'num_negative'] = np.sum(axis_data < 0)
        features[prefix + 'num_positive'] = np.sum(axis_data > 0)
        features[prefix + 'num_above_mean'] = np.sum(axis_data > np.mean(axis_data))
        features[prefix + 'skewness'] = skew(axis_data)
        features[prefix + 'kurtosis'] = kurtosis(axis_data)
        
    # Atribute în regim de frecvență pentru fiecare axă
    for i, axis in enumerate(['x', 'y', 'z']):
        try:
            axis_data = window[i].astype(np.float64)  # Convertiți în float64 înainte de a aplica transformata Fourier
            freq_domain = np.fft.rfft(axis_data)
            prefix = f'freq_axis_{i}_'
            
            features[prefix + 'mean'] = np.mean(freq_domain)
            features[prefix + 'std'] = np.std(freq_domain)
            features[prefix + 'mad'] = np.mean(np.abs(freq_domain - np.mean(freq_domain)))
            features[prefix + 'min'] = np.min(freq_domain)
            features[prefix + 'max'] = np.max(freq_domain)
            features[prefix + 'range'] = np.max(freq_domain) - np.min(freq_domain)
            features[prefix + 'median'] = np.median(freq_domain)
            features[prefix + 'mad_from_median'] = np.mean(np.abs(freq_domain - np.median(freq_domain)))
            features[prefix + 'iqr'] = iqr(freq_domain)
            features[prefix + 'num_negative'] = np.sum(freq_domain < 0)
            features[prefix + 'num_positive'] = np.sum(freq_domain > 0)
            features[prefix + 'num_above_mean'] = np.sum(freq_domain > np.mean(freq_domain))
            features[prefix + 'skewness'] = skew(freq_domain)
            features[prefix + 'kurtosis'] = kurtosis(freq_domain)

        except Exception as e:
            print(f"Error encountered while processing axis {i}: {e}")

    return features

# Alegeți o fereastră din setul de date de antrenament pentru a testa funcția extract_features
sample_window = racket_train_x[0]

# Apelați funcția extract_features și afișați rezultatele
extracted_features = extract_features(sample_window)
print(extracted_features)